<a href="https://colab.research.google.com/github/EdenKantor/Cloud-Computing-Course-Tut/blob/main/Cloud_Tut4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import pandas as pd
import requests
import plotly.graph_objects as go
from ipywidgets import widgets, Output, VBox, Tab
from IPython.display import display, clear_output

# --- שליפת נתונים מהמאגר הממשלתי ---
url = 'https://data.gov.il/api/3/action/datastore_search?resource_id=053cea08-09bc-40ec-8f7a-156f0677aff3&limit=10000'
response = requests.get(url)
data = response.json()
data_df = pd.DataFrame(data['result']['records'])

# --- טאב 1: סטטיסטיקות כלליות ---
def create_statistics_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>סטטיסטיקות כלליות</h3>"))

        # המרה בטוחה של עמודת השנה למספרים
        data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

        total_records = len(data_df)
        unique_makers = data_df['tozeret_nm'].nunique()
        unique_models = data_df['kinuy_mishari'].nunique()
        years = data_df['shnat_yitzur'].dropna().unique()

        stats_html = f"""
        <p><b>סך הרשומות:</b> {total_records}</p>
        <p><b>יצרנים ייחודיים:</b> {unique_makers}</p>
        <p><b>דגמים ייחודיים:</b> {unique_models}</p>
        <p><b>שנות ייצור:</b> {', '.join(map(str, sorted(map(int, years))))}</p>
        """
        display(widgets.HTML(stats_html))
    return output

# --- טאב 2: טבלה של המידע ---
def create_table_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>טבלת נתונים מלאה</h3>"))
        display(data_df.head(20))  # מציג את 20 השורות הראשונות
    return output

# --- טאב 3: גרף לפי שנות ייצור ---
def create_graph_tab():
    output = Output()
    with output:
        clear_output(wait=True)
        display(widgets.HTML("<h3>מספר כלי רכב לפי שנת ייצור</h3>"))

        # המרה בטוחה של עמודת השנה למספרים
        data_df['shnat_yitzur'] = pd.to_numeric(data_df['shnat_yitzur'], errors='coerce')

        # סינון נתונים תקינים
        years_count = data_df.groupby('shnat_yitzur').size().reset_index(name='count')
        years_count = years_count.dropna()  # הסרת שורות עם ערכים חסרים

        # יצירת גרף עמודות עם Plotly
        fig = go.Figure(
            data=[go.Bar(
                x=years_count['shnat_yitzur'],
                y=years_count['count'],
                marker_color='rgb(58,200,225)'
            )]
        )

        fig.update_layout(
            title="מספר כלי רכב לפי שנת ייצור",
            xaxis_title="שנה",
            yaxis_title="מספר כלי רכב",
            bargap=0.2
        )

        # הצגת הגרף
        fig.show()

    return output

# --- טאב 4: סינון יצרן, דגם ורמת גימור ---
def create_filter_tab():
    output = Output()
    tozeret_dropdown = widgets.Dropdown(
        options=['All'] + list(data_df['tozeret_nm'].dropna().unique()),
        description='יצרן:'
    )
    kinuy_dropdown = widgets.Dropdown(description='דגם:')
    trim_output = Output()

    def update_models(change):
        chosen_maker = tozeret_dropdown.value
        if chosen_maker == 'All':
            kinuy_dropdown.options = ['All']
        else:
            models = data_df[data_df['tozeret_nm'] == chosen_maker]['kinuy_mishari'].dropna().unique()
            kinuy_dropdown.options = ['All'] + list(models)

    def update_trim_levels(change):
        chosen_maker = tozeret_dropdown.value
        chosen_model = kinuy_dropdown.value
        with trim_output:
            clear_output(wait=True)
            if chosen_maker == 'All' or chosen_model == 'All':
                display(widgets.HTML("<p>אנא בחר יצרן ודגם להצגת פרטים.</p>"))
            else:
                filtered = data_df[(data_df['tozeret_nm'] == chosen_maker) & (data_df['kinuy_mishari'] == chosen_model)]
                trim_levels = filtered['ramat_gimur'].dropna().unique()
                total_records = len(filtered)
                display(widgets.HTML(f"<p><b>סך רשומות:</b> {total_records}</p>"))
                display(widgets.HTML(f"<p><b>רמות גימור ייחודיות:</b> {', '.join(trim_levels)}</p>"))

    tozeret_dropdown.observe(update_models, names='value')
    kinuy_dropdown.observe(update_trim_levels, names='value')

    with output:
        display(VBox([tozeret_dropdown, kinuy_dropdown, trim_output]))
    return output

# --- יצירת הטאבים ---
tab = Tab()
tab_children = [
    create_statistics_tab(),
    create_table_tab(),
    create_graph_tab(),
    create_filter_tab()
]
tab.children = tab_children
tab.set_title(0, "סטטיסטיקות")
tab.set_title(1, "טבלת נתונים")
tab.set_title(2, "גרף")
tab.set_title(3, "סינון")

# --- הצגת הטאבים ---
display(tab)